In [9]:
from fdp.db import query


In [11]:
from IPython.display import Markdown
import pandas as pd
import altair as alt

_ = alt.renderers.set_embed_options(theme="ggplot2", actions=False)


In [13]:
storage_provider_power_df = query("from filecoin_storage_providers_power").df()
storage_provider_power_df["date"] = pd.to_datetime(
    storage_provider_power_df["date"])
storage_provider_power_df.sample(5)


,date,provider_id,raw_power_bytes,raw_power_pibs,quality_adjusted_power_bytes,quality_adjusted_power_pibs,verified_data_power_bytes,verified_data_power_pibs,raw_power_pibs_bucket,verified_data_power_pibs_bucket
2053136,2022-06-13,f0113331,268555715084288,0.238525,269098450583552,0.239007,6.030394e+10,0.000054,>1<10,<0.1
1866723,2022-05-08,f01293938,959083377065984,0.851837,959083377065984,0.851837,0.000000e+00,0.000000,>1<10,=0
560636,2021-07-19,f01095558,1423592680062980,1.264404,1423592680062980,1.264404,0.000000e+00,0.000000,>10<50,=0
2884248,2022-11-11,f01214450,1111503176466430,0.987213,1111503176466430,0.987213,0.000000e+00,0.000000,>1<10,=0
4794159,2023-09-16,f01394448,1004025914851330,0.891754,1004025914851330,0.891754,0.000000e+00,0.000000,>1<10,=0


In [14]:
# | output: asis

print(
    f""" 
Updated on {storage_provider_power_df['date'].max().strftime('%Y-%m-%d')}.
"""
)


 
Updated on 2023-09-27.



In [15]:
latest_state = storage_provider_power_df[
    storage_provider_power_df["date"] == storage_provider_power_df["date"].max(
    )
]


::: {layout="[ [1,1,1]]"}

In [16]:
# | output: asis

unique_miners_with_active_power = latest_state[
    latest_state["raw_power_pibs"] > 0
].nunique()["provider_id"]

print(
    f"""
### Active Storage Providers
{unique_miners_with_active_power}
"""
)



### Active Storage Providers
3410



In [17]:
# | output: asis

print(
    f"""
### Total Raw Power
{latest_state['raw_power_pibs'].sum():.2f} PiBs
"""
)



### Total Raw Power
10731.36 PiBs



In [18]:
# | output: asis

print(
    f"""
### Total Quality Adjusted PiB Power
{latest_state['quality_adjusted_power_pibs'].sum():.2f} PiBs
"""
)



### Total Quality Adjusted PiB Power
24769.40 PiBs



:::

::: {layout="[ [1,1], [1] ]"}


In [19]:
# | output: true

display(Markdown(f"### Storage Provider Raw PiB Power"))

c = (
    alt.Chart(
        storage_provider_power_df.groupby(
            "date").sum()["raw_power_pibs"].reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("raw_power_pibs:Q", title="Raw PiB Power"),
        tooltip=["date", "raw_power_pibs"],
        color=alt.Color("raw_power_pibs_bucket:N",
                        title="Raw PiB Power Bucket"),
    )
    .properties(width="container")
)

c.display()


### Storage Provider Raw PiB Power

alt.Chart(...)

In [20]:
# | output: true

display(Markdown(f"### Storage Provider Quality Adjusted PiB Power"))

c = (
    alt.Chart(
        storage_provider_power_df.groupby("date")
        .sum()["quality_adjusted_power_pibs"]
        .reset_index()
    )
    .mark_area()
    .encode(
        x=alt.X("stat_date:T", title="Date"),
        y=alt.Y("quality_adjusted_pib_power:Q",
                title="Quality Adjusted PiB Power"),
        tooltip=["stat_date", "quality_adjusted_pib_power"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
)

c.display()


### Storage Provider Quality Adjusted PiB Power

ValueError: Unable to determine data type for the field "stat_date"; verify that the field name is not misspelled. If you are referencing a field from a transform, also confirm that the data type is specified correctly.

alt.Chart(...)

In [ ]:
# | output: true

display(Markdown(f"### Storage Providers With Power over Time"))

c = (
    alt.Chart(
        storage_provider_power_df[storage_provider_power_df["raw_power_pibs"] > 0]
        .groupby("date")
        .agg({"miner_id": "nunique"})
        .reset_index()
        .rename(columns={"stat_date": "date", "miner_id": "unique_miners"})
    )
    .mark_area()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("unique_miners:Q", title="Unique Miners"),
        tooltip=["date", "unique_miners"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
)

c.display()


### Storage Providers With Power over Time

alt.Chart(...)

:::

::: {layout="[ [1,1], [1] ]"}

In [12]:
# | output: true

display(
    Markdown(f"### Storage Provider Raw PiB Power Growth Rate (7 Days Rolling Mean)")
)

daily_pib_power = (
    storage_provider_power_df.groupby("stat_date").sum()[
        "raw_pib_power"].reset_index()
)

daily_pib_power = daily_pib_power.assign(
    growth_rate=lambda df: df["raw_pib_power"].pct_change(
        periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_pib_power)
    .mark_line()
    .encode(
        x=alt.X("stat_date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Raw PiB Power Growth Rate"),
        tooltip=["stat_date", "growth_rate"],
        color=alt.value("#55595c"),
    )
    .properties(width="container")
    .interactive(bind_y=False)
)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


### Storage Provider Raw PiB Power Growth Rate (7 Days Rolling Mean)

KeyError: 'stat_date'

In [13]:
# | output: true

display(
    Markdown(
        f"### Storage Providers With Power over Time Growth Rate (7 Days Rolling Mean)"
    )
)

daily_miners_with_power = (
    storage_provider_power_df[storage_provider_power_df["raw_byte_power"] > 0]
    .groupby("stat_date")
    .agg({"miner_id": "nunique"})
    .reset_index()
    .rename(columns={"stat_date": "date", "miner_id": "unique_miners"})
)

daily_miners_with_power = daily_miners_with_power.assign(
    growth_rate=lambda df: df["unique_miners"].pct_change(
        periods=7).rolling(7).mean()
    * 100
)

c = (
    alt.Chart(daily_miners_with_power)
    .mark_bar()
    .encode(
        x=alt.X("date:T", title="Date"),
        y=alt.Y("growth_rate:Q", title="Unique Miners Growth Rate"),
        tooltip=["date", "growth_rate"],
        # Color red if negative, green if positive
        color=alt.condition(
            alt.datum.growth_rate > 0,
            alt.value("#28a745"),  # The positive color
            alt.value("#dc3545"),  # The negative color
        ),
    )
    .properties(width="container")
    # Display line at x axis 0
).interactive(bind_y=False)

line = (
    alt.Chart(pd.DataFrame({"y": [0]}))
    .mark_rule()
    .encode(y="y", color=alt.value("#999"))
)

(c + line).display()


### Storage Providers With Power over Time Growth Rate (7 Days Rolling Mean)

alt.LayerChart(...)

:::